In [47]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import yfinance as yf 
import os

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras import losses
from tensorflow.keras import metrics
from keras.layers import LSTM
from tqdm import tqdm

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# from functions_forecasting import *

In [48]:
# data
ticker = 'ANF'
start_date = '2010-01-02'
end_date = '2019-12-31'
n_lags = 3

valid_size = 12
test_size = 12
batch_size = 3
n_epochs = 1000

In [49]:
df = yf.download(ticker,
    start = start_date,
    end = end_date,
    progress = False)

df = df.resample('M').last()
prices = df['Adj Close'].values.reshape(-1, 1)

In [ ]:
def split_sequence(sequence, n_steps_in, n_steps_out):
    '''
    Function for transforming time series into input acceptable by multilayer perceptron.abs

    Parameters
    ----------
    series : np.array
        Array of features 
    n_steps_in : int
        The number of lagged observations to consider as features

    Returns 
    -------
    X : np.array 
        Array of features 
    Y : np.array 
        Array of features
    '''

    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        X.append(sequence[i:end_ix])
        y.append(sequence[end_ix:out_end_ix])
    return np.array(X), np.array(y)


In [ ]:
minmax = MinMaxScaler(feature_range=(0, 1))

prices_train = prices[:-valid_size]
prices_valid = prices[-valid_size:]

minmax.fit(prices_train)

prices_train = minmax.transform(prices_train)
prices_valid = minmax.transform(prices_valid)

prices_scaled = np.concatenate((prices_train, 
                                prices_valid)).flatten()
plt.plot(prices_scaled)

In [ ]:
X, y = split_sequence(prices_scaled, 3,1)
X = X.reshape(X.shape[0], X.shape[1], 1)
y = y.reshape(y.shape[0], 1)

In [ ]:
X.shape

In [ ]:
X_train = X[:-valid_size]
y_train = y[:-valid_size]

X_val = X[-valid_size:]
y_val = y[-valid_size:]
# val_data = np.array((X_val, y_val))

In [ ]:
X_train.shape

In [ ]:
class RNN(tf.keras.Model):

    def __init__(self,steps_in,steps_out):
        super(RNN, self).__init__()
        self.LSTM_layer1 = tf.keras.layers.LSTM(256, return_sequences=True, input_shape = (steps_in,1))
        # self.drop_out_layer_1 = tf.keras.layers.Dropout(0.2)
        # self.LSTM_layer2 = tf.keras.layers.LSTM(256, return_sequences=True)
        # self.drop_out_layer_2 = tf.keras.layers.Dropout(0.2)
        self.dense_layer3 = tf.keras.layers.Dense(steps_out)

    # In the call function, we define the Model's forward pass.
    def build(self):
        model = tf.keras.Sequential([
        # Layer 1: 
        self.LSTM_layer1,
        # self.drop_out_layer_1,
        # Layer 2:
        # self.LSTM_layer2,
        # self.drop_out_layer_2,
        # Layer 3:
        self.dense_layer3 
        ])

        return model

In [ ]:
# model = RNN(3,1).build()
# # optimizer = tf.keras.optimizers.Adam(0.0001)
# model(X_train[0:1])
# # model.summary()

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(TimeStep, 1)))
model.add(Dense(1))
# model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])

In [ ]:
model(X_train[0:1])

In [16]:
mse = tf.keras.losses.MeanSquaredError()

In [17]:
@tf.function
def train_step(x, y): 
    with tf.GradientTape() as tape:

        y_hat = model(x) 

        loss = mse(y, y_hat) 

        
        grads = tape.gradient(loss, model.trainable_variables) 

        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

In [18]:
history = []
history_val = []
losses_val = []

for epoch in tqdm(range(5)):
    for i in range(len(X_train)):
        # Grab a batch and propagate it through the network
        x_batch, y_batch = X_train[i:i+1], y_train[i:i+1]
        loss = train_step(x_batch, y_batch)

        # Update the progress bar
        history.append(loss.numpy().mean())

        # Update validation loss history
        for j in range(len(X_val)):
            y_hat = model(X_train[j].reshape(1,batch_size))
            losses_val.append( mse(y_hat, y_val[j].reshape(1,1)).numpy() )
            
        history_val.append(np.asarray(losses_val).mean())
        losses_val = []
      
model.save_weights(checkpoint_prefix)

0%|          | 0/5 [00:02<?, ?it/s]


ValueError: Input 0 of layer sequential is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (1, 3)